In [2]:
import pandas as pd

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv("SmartFix_Chatbox_UserInteraction_Dataset - Sheet1.csv")

In [5]:
df.head()

,Interaction_id,Session_id,Time_Stamp,User_Input,detected_intent,entities,chatbot_response,confidence_score,language,handover_flag,feedback_rating
0,nt001,sess001,2025-07-11 11:00,My AC is not cooling,ReportApplianceIssue,"appliance: AC, issue: cooling problem",Looks like you need an AC technician. Shall I ...,0.85,en,False,5
1,int002,sess002,2025-07-11 11:05,My geyser isn't heating,ReportApplianceIssue,"appliance: geyser, issue: heating problem",Try checking the thermostat. Want me to walk y...,0.88,en,False,4
2,int003,sess003,2025-07-11 11:10,My fan is making a clicking sound,TroubleshootAppliance,"appliance: fan, issue: noise",Check if the blades are loose or if something ...,0.83,en,False,5
3,int004,sess004,2025-07-11 11:12,My washing machine won’t drain,ReportApplianceIssue,"appliance: washing machine, issue: drainage issue",The drain hose might be blocked. Would you lik...,0.87,en,False,5
4,int005,sess005,2025-07-11 11:16,My microwave stopped mid-cycle,ReportApplianceIssue,"appliance: microwave, issue: sudden stop",It could be a fuse issue. Unplug for 10 minute...,0.82,en,False,4


In [6]:
df.describe()

,confidence_score,feedback_rating
count,15.000000,15.000000
mean,0.841333,4.400000
std,0.030675,0.736788
min,0.790000,3.000000
25%,0.820000,4.000000
50%,0.840000,5.000000
75%,0.865000,5.000000
max,0.890000,5.000000


In [7]:
intent_counts = df['detected_intent'].value_counts()
print(intent_counts)

detected_intent
ReportApplianceIssue     10
TroubleshootAppliance     5
Name: count, dtype: int64


# DATA PREPROCESSING

### Convert text to lowercase

In [8]:
import pandas as pd
import nltk
import string

from nltk.corpus import stopwords   # used for removing stopwords like , . ! 
from nltk.tokenize import word_tokenize # used for tokenizing sentences into words to identify problem.

In [9]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')



[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\USER\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [10]:
import nltk
nltk.download('punkt', download_dir='nltk_data')
nltk.download('stopwords', download_dir='nltk_data')
nltk.data.path.append('nltk_data')


[nltk_data] Downloading package punkt to nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [11]:
# making a function to preprocess the text data, like removing punctuation, converting to lowercase, and removing stopwords.


def preprocess(text):
    # Convert to lowercase
    text = text.lower()
    
    # Tokenize into words
    tokens = word_tokenize(text)
    
    # Remove punctuation and stopwords
    cleaned = [word for word in tokens if word not in string.punctuation and word not in stopwords.words('english')]
    
    # Return cleaned text
    return " ".join(cleaned)


In [12]:
print(df.columns)

Index(['Interaction_id', 'Session_id', 'Time_Stamp', 'User_Input',
       'detected_intent', 'entities', 'chatbot_response', 'confidence_score',
       'language', 'handover_flag', 'feedback_rating'],
      dtype='object')


In [13]:
df['cleaned_input'] = df['User_Input'].apply(preprocess)
print(df[['User_Input', 'cleaned_input']].head())


                          User_Input                cleaned_input
0               My AC is not cooling                   ac cooling
1            My geyser isn't heating           geyser n't heating
2  My fan is making a clicking sound    fan making clicking sound
3     My washing machine won’t drain      washing machine ’ drain
4     My microwave stopped mid-cycle  microwave stopped mid-cycle


# Training Machine learning model

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report


In [15]:
X = df['cleaned_input']           # Features (text)
y = df['detected_intent']         # Labels (intent)


converting  text into numbers that the model can understand.





In [16]:
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)


Split into Training and Test Sets

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_vectorized, y, test_size=0.2, random_state=42)


Train the Model

In [18]:
model = MultinomialNB()
model.fit(X_train, y_train)


,alpha,1.0
,force_alpha,True
,fit_prior,True
,class_prior,None


Make Predictions and Evaluate

In [19]:
y_pred = model.predict(X_test)

print(classification_report(y_test, y_pred))


                      precision    recall  f1-score   support

ReportApplianceIssue       1.00      1.00      1.00         3

            accuracy                           1.00         3
           macro avg       1.00      1.00      1.00         3
        weighted avg       1.00      1.00      1.00         3



In [20]:
model.predict(vectorizer.transform(["My AC is leaking again"]))


array(['ReportApplianceIssue'], dtype='<U21')